Evaluating the error individually and with the difference from sharpening GOES images from bands 7 and 14 using laplace and unsharpmask sharpening.

In [6]:
import os
import random

import numpy as np
import pandas as pd

from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

random.seed(10)

In [3]:
def control(path, file):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    mae = mean_absolute_error(img, control)
    rmse = mean_squared_error(img, control, squared=False)
    return {'File': file, 'Band': file[19:21], 'Control_MAE': mae, 'Control_RMSE': rmse}

def laplace_sharpening(path, file):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'File': file, 'Band': file[19:21], 'LP_MAE': mae, 'LP_RMSE': rmse}

def unsharpmask_sharpening(path, file):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'File': file, 'Band': file[19:21], 'UM_MAE': mae, 'UM_RMSE': rmse}

In [29]:
def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    prefix = file[:19] + band + file[21:41]
    for x in os.listdir(path):
        if x.startswith(prefix):
            return x

def update_dict(path, file, data):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    newdata = {}
    newdata.update(control(path, file))
    newdata.update(laplace_sharpening(path, file))
    newdata.update(unsharpmask_sharpening(path, file))
    data.update(newdata)

## Evaluating Error

In [90]:
path = '../../GOES_Files/npy_files/'
data = []

for i in range (20):
    file07 = ''
    while file07[19:21] != '07':
        file07 = random.choice(os.listdir(path))
    file14 = find_npy_file(path, file07, '14')

    data07 = {}
    update_dict(path, file07, data07)
    
    data14 = {}
    update_dict(path, file14, data14)

    data.append(data07)
    data.append(data14)
    
df = pd.DataFrame(data)

In [104]:
print("Mean Error")
df.groupby(df['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 
                            'LP_MAE': 'mean', 'LP_RMSE': 'mean', 
                            'UM_MAE': 'mean', 'UM_RMSE': 'mean'})

Mean Error


,Control_MAE,Control_RMSE,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE
Band,,,,,,
07,0.667012,0.967031,0.540472,0.798883,0.552541,0.807855
14,0.757626,1.158540,0.614688,0.958806,0.628296,0.970988


In [103]:
print("Standard Deviation of Error")
df.groupby(df['Band']).agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 
                            'LP_MAE': 'std', 'LP_RMSE': 'std', 
                            'UM_MAE': 'std', 'UM_RMSE': 'std'})

Standard Deviation of Error


,Control_MAE,Control_RMSE,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE
Band,,,,,,
07,0.14802,0.225049,0.118500,0.185029,0.119201,0.184288
14,0.19627,0.298229,0.157038,0.245976,0.159503,0.247765


In [100]:
print("Difference Mean Error")
diffdf.agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 
            'LP_MAE': 'mean', 'LP_RMSE': 'mean', 
            'UM_MAE': 'mean', 'UM_RMSE': 'mean'})

Difference Mean Error


Control_MAE    -0.090614
Control_RMSE   -0.191508
LP_MAE         -0.074216
LP_RMSE        -0.159924
UM_MAE         -0.075755
UM_RMSE        -0.163133
dtype: float64

In [102]:
print("Difference Standard Deviation of Error")
diffdf.agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 
            'LP_MAE': 'std', 'LP_RMSE': 'std', 
            'UM_MAE': 'std', 'UM_RMSE': 'std'})

Difference Standard Deviation of Error


Control_MAE     0.079142
Control_RMSE    0.140882
LP_MAE          0.062793
LP_RMSE         0.114770
UM_MAE          0.064056
UM_RMSE         0.115929
dtype: float64